# Purpose of this notebook:

- ETL for our initial Shakespeare dataset
- Creation of src and target examples for 

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast
import json
import random
import re

In [2]:
random.seed(102)

# Creating tokenizer

In [3]:
with open("./data/source/shakespeare_tokenize.txt", 'r') as file:
    ss = file.read()

In [4]:
ss = str(ss).lower().replace('\n\n','\n').replace('  ', ' ').replace(';', '').replace(':', '').replace('[', '').replace(']', '').replace('  ', ' ')

In [5]:
ss_list = ss.split('\n')
ss_list = [i.strip() for i in ss_list]
ss_list[:5]

["a midsummer-night's dream",
 'now , fair hippolyta , our nuptial hour',
 'draws on apace four happy days bring in',
 'another moon but o ! methinks how slow',
 'this old moon wanes she lingers my desires ,']

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = tokenizer.train_new_from_iterator(iter(ss_list),32000)
tokenizer.vocab_size

30267

In [7]:
tokenizer.tokenize("my tongue should catch your tongue's sweet melody ."), tokenizer.tokenize("demetrius , i'll avouch it to his head ,")

(['my',
  'tongue',
  'should',
  'catch',
  'your',
  'tongue',
  "'",
  's',
  'sweet',
  'melody',
  '.'],
 ['demetrius', ',', 'i', "'", 'll', 'avouch', 'it', 'to', 'his', 'head', ','])

In [8]:
tokenizer_save_path = 'shakespeare-tokenizer-bert/plays'
tokenizer.save_pretrained(tokenizer_save_path)

('shakespeare-tokenizer-bert/plays\\tokenizer_config.json',
 'shakespeare-tokenizer-bert/plays\\special_tokens_map.json',
 'shakespeare-tokenizer-bert/plays\\vocab.txt',
 'shakespeare-tokenizer-bert/plays\\added_tokens.json',
 'shakespeare-tokenizer-bert/plays\\tokenizer.json')

# Exploratory

In [12]:
df = pd.read_csv('data/source/shakespeare.csv')

In [13]:
df.head(5)

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [14]:
# df['Play'].str.lower().unique()

In [15]:
# df['Player'].str.lower().unique()

In [16]:
# french characters: alencon, alice, king of france, katharine, dauphin
# french speaking characters

In [17]:
# df[df['Player'].str.lower() == 'katharine']
# maybe could remove the whole of Henry V since that is where most french lines are found

In [18]:
# Remove most contextual lines (act numbers, leaving, end, etc)
remove = df[(df['Player'].notna()) & (df["PlayerLine"]!='Exeunt')]
data = remove[['Play', 'PlayerLinenumber', "Player", "PlayerLine"]]
data.head(5)

,Play,PlayerLinenumber,Player,PlayerLine
3,Henry IV,1.0,KING HENRY IV,"So shaken as we are, so wan with care,"
4,Henry IV,1.0,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,Henry IV,1.0,KING HENRY IV,And breathe short-winded accents of new broils
6,Henry IV,1.0,KING HENRY IV,To be commenced in strands afar remote.
7,Henry IV,1.0,KING HENRY IV,No more the thirsty entrance of this soil


# Preprocessing Shakespeare

In [30]:
class ShakespeareData:
    """Creates a cleaned up version of the shakespeare dataset"""
    def __init__(self, filepath='data/source/shakespeare.csv'):        
        data = pd.read_csv(filepath)
        self.data = self.clean(data) 
            
    def clean(self, data):
        repl = {
            '@\w*': ' ', 
            '&amp;' : 'and',
            '&#\w*;': ' ', 
            '\\n\.':' ' ,
            '\\n':' ',
            "\.{2,}": '.', 
            "!{2,}":'!', 
            "\?{2,}":'?', 
            '_':" ",
            ' +': ' ', 
            '\-{2,}': ' ',
            ':': '',
            ';': '',
            '\[': '',
            '\]': '',
            '  ': ' '
            }

        data = data[(data['Player'].notna()) & (data["PlayerLine"]!='Exeunt') & (data["Play"]!='Henry V') & (data['ActSceneLine'].notna())]
        data = data[['ActSceneLine', "Player", "PlayerLine", "Play", "PlayerLinenumber"]]
        data['Player'] = data['Player'].str.lower()
        data['ActScene'] = data['ActSceneLine'].apply(extract_before_second_full_stop)
        cleaned = data['PlayerLine'].apply(lambda x: x.strip().lower())
        cleaned = cleaned.replace(repl, regex=True)
        cleaned_data = data.assign(text = cleaned)
        return cleaned_data

def extract_before_second_full_stop(string):
    """ Finds the act number"""
    index = string.find('.', string.find('.') + 1)  # Find index of second full stop
    if index != -1:
        return string[:index]
    else:
        return string

In [31]:
def generate_src_trg_dataset(text_col):
    data = []
    
    for i in range(len(text_col)-1):
        dictionary = {}
        dictionary["src"] = text_col.iloc[i]
        dictionary["trg"] = text_col.iloc[i+1]
        data.append(dictionary)

#     for i in range(len(text_col)-5):
#         dictionary = {}
#         dictionary["src"] = (text_col.iloc[i] + " " + text_col.iloc[i+1] + " " + text_col.iloc[i+2]).strip()
#         dictionary["trg"] = (text_col.iloc[i+3] + " " + text_col.iloc[i+4] + " " + text_col.iloc[i+5]).strip()
#         data.append(dictionary)
    
    return data

def save_src_trg_dataset(data, filename = 'data', folder_dir = ''):
    fn = folder_dir + filename + '.jsonl'
    if not os.path.exists(folder_dir):
        os.mkdir(folder_dir)
    with open(fn, 'w') as outfile:
        for entry in data:
            json.dump(entry, outfile)
            outfile.write('\n')

def generate_and_save_dataset(data_text_column, filename= 'data', folder_dir= ''):
    data = generate_src_trg_dataset(data_text_column)
    
    save_src_trg_dataset(data, filename=filename, folder_dir=folder_dir)

In [32]:
# with open('vocab_list.pickle', 'wb') as handle:
#     pickle.dump(vocab.idx_word, handle)

In [33]:
shakespeare_data = ShakespeareData()

In [34]:
shakespeare_data.data.head() # 107459 -> # 101919 after removing na in actsceneline

,ActSceneLine,Player,PlayerLine,Play,PlayerLinenumber,ActScene,text
3,1.1.1,king henry iv,"So shaken as we are, so wan with care,",Henry IV,1.0,1.1,"so shaken as we are, so wan with care,"
4,1.1.2,king henry iv,"Find we a time for frighted peace to pant,",Henry IV,1.0,1.1,"find we a time for frighted peace to pant,"
5,1.1.3,king henry iv,And breathe short-winded accents of new broils,Henry IV,1.0,1.1,and breathe short-winded accents of new broils
6,1.1.4,king henry iv,To be commenced in strands afar remote.,Henry IV,1.0,1.1,to be commenced in strands afar remote.
7,1.1.5,king henry iv,No more the thirsty entrance of this soil,Henry IV,1.0,1.1,no more the thirsty entrance of this soil


In [40]:
comedies = ['As you like it', 'A Midsummer nights dream', 'Twelfth Night', 'Much Ado about nothing', 'Taming of the Shrew', 'A Comedy of Errors', 'Loves Labours Lost', 'Merchant of Venice', 'Measure for measure', 'Alls well that ends well']
comedies_only = shakespeare_data.data[shakespeare_data.data['Play'].isin(comedies)]
comedies_only.head()

,ActSceneLine,Player,PlayerLine,Play,PlayerLinenumber,ActScene,text
12663,1.1.1,countess,"In delivering my son from me, I bury a second ...",Alls well that ends well,1.0,1.1,"in delivering my son from me, i bury a second ..."
12664,1.1.2,bertram,"And I in going, madam, weep o'er my father's d...",Alls well that ends well,2.0,1.1,"and i in going, madam, weep o'er my father's d..."
12665,1.1.3,bertram,"anew: but I must attend his majesty's command, to",Alls well that ends well,2.0,1.1,"anew but i must attend his majesty's command, to"
12666,1.1.4,bertram,"whom I am now in ward, evermore in subjection.",Alls well that ends well,2.0,1.1,"whom i am now in ward, evermore in subjection."
12667,1.1.5,lafeu,"You shall find of the king a husband, madam, you,",Alls well that ends well,3.0,1.1,"you shall find of the king a husband, madam, you,"


In [36]:
def split_long_lines(line, max_length=200):
    # Split long lines in the data set in order to fit sequence length.
    if len(line) <= max_length:
        return [line]

    # Find best line break area, if not, 
    match_punctuation = re.search(r'[\.,;:!?>]\s*', line[:max_length][::-1])
    # match_word = re.search(r'\s', line[:max_length][::-1])
    if match_punctuation:
        split_point = max_length-match_punctuation.start()
#         elif match_word:
#             split_point = max_length - match_word.start()
    else:
        split_point = max_length

    # Split lines and recurse on unsplit part
    return [line[:split_point].strip()] + split_long_lines(line[split_point:].strip(), max_length=max_length)
    
def remove_separator(lines):
    return [i.replace(">", "").strip().replace("  ", " ") for i in lines]

In [41]:
# Aggregate character lines into a single line
data_per_convo = (
    shakespeare_data.data.groupby(['Play', 'Player', 'ActScene', 'PlayerLinenumber'],sort=False)['text']
    .agg(lambda x: ' > '.join(x)).reset_index()
)
# Split long single actor lines into sequence length-ish length
data_per_convo['text_processed']=data_per_convo['text'].apply(split_long_lines)
data_per_convo['text_processed']=data_per_convo['text_processed'].apply(remove_separator)
data_per_convo['text_processed'].iloc[0][:5]

# Create processed data for each row
data_per_convo = data_per_convo.explode('text_processed')

In [42]:
data_per_convo['text_processed_with_player'] = data_per_convo['Player'] + " : " + data_per_convo['text_processed']
data_per_convo['text_processed_with_player'].head()

0    king henry iv : so shaken as we are, so wan wi...
0    king henry iv : no more the thirsty entrance o...
0    king henry iv : of hostile paces those opposed...
0    king henry iv : and furious close of civil but...
0    king henry iv : like an ill-sheathed knife, no...
Name: text_processed_with_player, dtype: object

In [43]:
all_plays = data_per_convo.Play.unique()

In [44]:
all_plays

array(['Henry IV', 'Henry VI Part 1', 'Henry VI Part 2',
       'Henry VI Part 3', 'Alls well that ends well', 'As you like it',
       'Antony and Cleopatra', 'A Comedy of Errors', 'Coriolanus',
       'Cymbeline', 'Hamlet', 'Henry VIII', 'King John', 'Julius Caesar',
       'King Lear', 'Loves Labours Lost', 'macbeth',
       'Measure for measure', 'Merchant of Venice',
       'Merry Wives of Windsor', 'A Midsummer nights dream',
       'Much Ado about nothing', 'Othello', 'Pericles', 'Richard II',
       'Richard III', 'Romeo and Juliet', 'Taming of the Shrew',
       'The Tempest', 'Timon of Athens', 'Titus Andronicus',
       'Troilus and Cressida', 'Twelfth Night', 'Two Gentlemen of Verona',
       'A Winters Tale'], dtype=object)

In [41]:
# Aggregate character lines into a single line
data_per_convo_comedies = (
    comedies_only.groupby(['Play', 'Player', 'ActScene', 'PlayerLinenumber'],sort=False)['text']
    .agg(lambda x: ' > '.join(x)).reset_index()
)
# Split long single actor lines into sequence length-ish length
data_per_convo_comedies['text_processed']=data_per_convo_comedies['text'].apply(split_long_lines)
data_per_convo_comedies['text_processed']=data_per_convo_comedies['text_processed'].apply(remove_separator)
data_per_convo_comedies['text_processed'].iloc[0][:5]

# Create processed data for each row
data_per_convo_comedies = data_per_convo_comedies.explode('text_processed')

In [44]:
data_per_convo_comedies['text_processed_with_player'] = data_per_convo_comedies['Player'] + " : " + data_per_convo_comedies['text_processed']
data_per_convo_comedies['text_processed_with_player'].head()

0    countess : in delivering my son from me, i bur...
1    bertram : and i in going, madam, weep o'er my ...
2    lafeu : you shall find of the king a husband, ...
2    lafeu : worthiness would stir it up where it w...
3    countess : what hope is there of his majesty's...
Name: text_processed_with_player, dtype: object

### Preparing train/test Shakespeare data

In [54]:
processed_data = []
for play in all_plays:
    text_col = data_per_convo[data_per_convo['Play'] == play]['text_processed']
#     text_col = data_per_convo[data_per_convo['Play'] == play]['text_processed_with_player']
    processed_data += generate_src_trg_dataset(text_col)

In [56]:
text_col

28614    if you shall chance, camillo, to visit bohemia...
28615    i think, this coming summer, the king of sicil...
28616    wherein our entertainment shall shame us we wi...
28617                                         beseech you,
28618    verily, i speak it in the freedom of my knowle...
                               ...                        
29360    o, peace, paulina! thou shouldst a husband tak...
29360    for i saw her, as i thought, dead, and have in...
29360    and take her by the hand, whose worth and hone...
29360    that e'er i put between your holy looks my ill...
29360    lead us from hence, where we may leisurely eac...
Name: text_processed, Length: 1154, dtype: object

In [57]:
processed_data[:5]

[{'src': 'so shaken as we are, so wan with care, find we a time for frighted peace to pant, and breathe short-winded accents of new broils to be commenced in strands afar remote.',
  'trg': "no more the thirsty entrance of this soil shall daub her lips with her own children's blood, nor more shall trenching war channel her fields, nor bruise her flowerets with the armed hoofs"},
 {'src': "no more the thirsty entrance of this soil shall daub her lips with her own children's blood, nor more shall trenching war channel her fields, nor bruise her flowerets with the armed hoofs",
  'trg': 'of hostile paces those opposed eyes, which, like the meteors of a troubled heaven, all of one nature, of one substance bred, did lately meet in the intestine shock'},
 {'src': 'of hostile paces those opposed eyes, which, like the meteors of a troubled heaven, all of one nature, of one substance bred, did lately meet in the intestine shock',
  'trg': 'and furious close of civil butchery shall now, in mutua

In [58]:
random.shuffle(processed_data) # in-place shuffle

In [59]:
n = len(processed_data)
n

40664

In [60]:
train_cutoff = int(n*0.8)

In [61]:
# want to preserve order
ss_train_data = processed_data[:train_cutoff]
ss_test_data = processed_data[-10:]
ss_val_data = processed_data[train_cutoff:-10]

In [62]:
len(ss_test_data), len(ss_train_data), len(ss_val_data)

(10, 32531, 8123)

In [63]:
ss_train_data[:5]

[{'src': 'this is the very false gallop of verses why do you infect yourself with them?',
  'trg': 'peace, you dull fool! i found them on a tree.'},
 {'src': 'and fair men call for grace. aaron will have his soul black like his face.',
  'trg': 'o, here i lift this one hand up to heaven, and bow this feeble ruin to the earth if any power pities wretched tears, to that i call! what, wilt thou kneel with me? do, then, dear heart,'},
 {'src': 'miranda. o my father, i have broke your hest to say so!',
  'trg': "admired miranda! indeed the top of admiration! worth what's dearest to the world! full many a lady i have eyed with best regard and many a time the harmony of their tongues hath into bondage"},
 {'src': 'i have, thou gallant trojan, seen thee oft labouring for destiny make cruel way through ranks of greekish youth, and i have seen thee, as hot as perseus, spur thy phrygian steed,',
  'trg': "despising many forfeits and subduements, when thou hast hung thy advanced sword i' the air, 

In [64]:
save_src_trg_dataset(ss_train_data, filename= 'train', folder_dir= 'data/')

In [65]:
save_src_trg_dataset(ss_val_data, filename= 'valid', folder_dir= 'data/')

In [66]:
save_src_trg_dataset(ss_test_data, filename= 'test', folder_dir= 'data/')

### Preparing train/test Shakespeare data (Comedies only)

In [46]:
processed_data = []
for play in comedies:
    text_col = data_per_convo_comedies[data_per_convo_comedies['Play'] == play]['text_processed']
#     text_col = data_per_convo[data_per_convo['Play'] == play]['text_processed_with_player']
    processed_data += generate_src_trg_dataset(text_col)

In [47]:
text_col

0      in delivering my son from me, i bury a second ...
1      and i in going, madam, weep o'er my father's d...
2      you shall find of the king a husband, madam, y...
2      worthiness would stir it up where it wanted ra...
3         what hope is there of his majesty's amendment?
                             ...                        
934    let us from point to point this story know, to...
934    for i can guess that by thy honest aid thou ke...
934    and if it end so meet, the bitter past, more w...
935    the king's a beggar, now the play is done all ...
935    ours be your patience then, and yours our part...
Name: text_processed, Length: 1226, dtype: object

In [50]:
processed_data[:2]

[{'src': 'i would not have my right rosalind of this mind, for, i protest, her frown might kill me.',
  'trg': 'by this hand, it will not kill a fly. but come, now i will be your rosalind in a more coming-on disposition, and ask me what you will. i will grant it.'},
 {'src': 'i thought, by your readiness in the office, you had continued in it some time. you say, seven years together?',
  'trg': 'and a half, sir.'}]

In [49]:
random.shuffle(processed_data) # in-place shuffle

In [51]:
n = len(processed_data)
n

10883

In [52]:
train_cutoff = int(n*0.8)

In [53]:
# want to preserve order
ss_train_data = processed_data[:train_cutoff]
ss_test_data = processed_data[-10:]
ss_val_data = processed_data[train_cutoff:-10]

In [54]:
len(ss_test_data), len(ss_train_data), len(ss_val_data)

(10, 8706, 2167)

In [55]:
ss_train_data[:5]

[{'src': 'i would not have my right rosalind of this mind, for, i protest, her frown might kill me.',
  'trg': 'by this hand, it will not kill a fly. but come, now i will be your rosalind in a more coming-on disposition, and ask me what you will. i will grant it.'},
 {'src': 'i thought, by your readiness in the office, you had continued in it some time. you say, seven years together?',
  'trg': 'and a half, sir.'},
 {'src': 'that she is worthy, i know.',
  'trg': 'that i neither feel how she should be loved nor know how she should be worthy, is the opinion that fire cannot melt out of me i will die in it at the stake.'},
 {'src': "in stamps that are forbid 'tis all as easy falsely to take away a life true made as to put metal in restrained means to make a false one.",
  'trg': "'tis set down so in heaven, but not in earth."},
 {'src': 'music, ho! music, such as charmeth sleep!',
  'trg': "now, when thou wakest, with thine own fool's eyes peep."}]

In [56]:
save_src_trg_dataset(ss_train_data, filename= 'train', folder_dir= 'data/comedies_only')

In [57]:
save_src_trg_dataset(ss_val_data, filename= 'valid', folder_dir= 'data/comedies_only')

In [58]:
save_src_trg_dataset(ss_test_data, filename= 'test', folder_dir= 'data/comedies_only')

# Extracting only comedies

In [20]:
data['Play'].unique()

array(['Henry IV', 'Henry VI Part 1', 'Henry VI Part 2',
       'Henry VI Part 3', 'Alls well that ends well', 'As you like it',
       'Antony and Cleopatra', 'A Comedy of Errors', 'Coriolanus',
       'Cymbeline', 'Hamlet', 'Henry V', 'Henry VIII', 'King John',
       'Julius Caesar', 'King Lear', 'Loves Labours Lost', 'macbeth',
       'Measure for measure', 'Merchant of Venice',
       'Merry Wives of Windsor', 'A Midsummer nights dream',
       'Much Ado about nothing', 'Othello', 'Pericles', 'Richard II',
       'Richard III', 'Romeo and Juliet', 'Taming of the Shrew',
       'The Tempest', 'Timon of Athens', 'Titus Andronicus',
       'Troilus and Cressida', 'Twelfth Night', 'Two Gentlemen of Verona',
       'A Winters Tale'], dtype=object)

In [29]:
comedies_only = data[data['Play'].isin(comedies)]
comedies_only

,Play,PlayerLinenumber,Player,PlayerLine
12660,Alls well that ends well,9.0,KING EDWARD IV,ACT I
12661,Alls well that ends well,9.0,KING EDWARD IV,SCENE I. Rousillon. The COUNT's palace.
12662,Alls well that ends well,9.0,KING EDWARD IV,"Enter BERTRAM, the COUNTESS of Rousillon, HELE..."
12663,Alls well that ends well,1.0,COUNTESS,"In delivering my son from me, I bury a second ..."
12664,Alls well that ends well,2.0,BERTRAM,"And I in going, madam, weep o'er my father's d..."
...,...,...,...,...
105545,Twelfth Night,127.0,Clown,"A great while ago the world begun,"
105546,Twelfth Night,127.0,Clown,"With hey, ho, and c."
105547,Twelfth Night,127.0,Clown,"But that's all one, our play is done,"
105548,Twelfth Night,127.0,Clown,And we'll strive to please you every day.


In [28]:
comedies.count(data['Play'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().